# Протоколы аутентификации с нулевым разглашением

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел
from Crypto.Util import number

# КГПСЧ
from Crypto.Random import random

# Понадобится в протоколе Шнорра
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Hash import SHA256

# Немного математики
import gmpy2

# Для генерации идентификаторов клиентов
import uuid

---

In [4]:
def randint(a, b) -> gmpy2.mpz:
    """
    Генерация случайного числа в отрезке [a, b] с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_a = int(a)
    native_b = int(b)
    
    return gmpy2.mpz(random.randint(native_a, native_b))


def getrandbits(n) -> gmpy2.mpz:
    """
    Генерация случайного числа длиной n бит с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_n = int(n)
    
    return gmpy2.mpz(random.getrandbits(native_n))

----

## Протокол Фиата-Шамира
![Fiat-Shamir Protocol](./images/Fiat-Shamir_2.png)

In [5]:
class FiatShamirCA(object):
    PRIME_BITS = 1024
    
    def __init__(self):
        """
        Инициализация доверенного центра.
        Генерируются два простых числа и их произведение.
        """
        
        self._db = {}
        
        self._p = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._q = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._modulus = gmpy2.mul(self._p, self._q)
        
        trace('[FiatShamirCA]', f'Modulus = {self._modulus}')
        
        
    def register_client(self, client_id: uuid.UUID, client_public_key: gmpy2.mpz) -> None:
        """
        Регистрация клиента. На вход принимает идентификатор клиента и его открытый ключ.
        Ничего не возвращает.
        Выбрасывает исключение, если клиент уже существует.
        """
        
        trace('[FiatShamirCA]', f'''Attempting to register client {client_id} 
        with public key {client_public_key}''')
        
        if client_id in self._db:
            trace('[FiatShamirCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Теперь сохраняю открытый ключ
        #
        
        self._db[client_id] = client_public_key
        trace('[FiatShamirCA]', 'Client registered successfully')
        
        
    def get_public_key(self, client_id: uuid.UUID) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента. Получает на вход идентификатор клиента.
        Возвращает открытый ключ.
        Выбрасывает исключение, если пользователь отсутствует.
        """
        
        return self._db[client_id]
       
        
    @property
    def modulus(self) -> gmpy2.mpz:
        """
        Возвращает модуль.
        """
        
        return self._modulus

In [6]:
class FiatShamirClient(object):
    def __init__(self, modulus: gmpy2.mpz, *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz] = None):
        """
        Инициализация клиента с выработкой ключевой пары и идентификатора.
        Последний параметр требуются для демонстрации неудачной аутентификации.
        """
        
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        self._r = None
        
        self._private_key = FiatShamirClient._generate_coprime(modulus)
        self._public_key = gmpy2.powmod(self._private_key, 2, modulus) if fake_data is None else fake_data[1]
        
        trace('[FiatShamirClient]', f'Client {self._id}')
        trace('[FiatShamirClient]', f'Public key = {self._public_key}')
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора.
        """
        
        return self._id
    
    
    def get_x(self, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение случайного значения, вычисляемого по формуле:
                    x = r ** 2 mod n,
        где n - модуль, r - случайное целое число из отрезка [1, n - 1].
        Значение r сохраняется.
        """
        
        self._r = randint(1, modulus - 1)
        return gmpy2.powmod(self._r, 2, modulus)
    
    
    def get_y(self, e: gmpy2.mpz, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение числа, вычисляемого по формуле:
                    y = r * s ** e mod n,
        где n - модуль, s - закрытый ключ пользователя, r - сгенерированное 
        функцией get_x внутреннее значение, e - целое число из отрезка [0, 1],
        пришедшее от проверяющего.
        """
        
        product = gmpy2.mul(self._r, gmpy2.powmod(self._private_key, e, modulus))
        return gmpy2.t_mod(product, modulus)
        
        
    @staticmethod
    def _generate_coprime(modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Генерация случайного числа из отрезка [1, n - 1], взаимно
        простого с n, где n - модуль.
        """
        
        #
        # Генерирую число до тех пор, пока оно не станет взаимно простым с модулем
        #
        
        result = randint(1, modulus - 1)
        while gmpy2.gcd(result, modulus) != 1:
            result = randint(1, modulus - 1)
            
        return result

In [7]:
class FiatShamirVerifier(object):
    def __init__(self, iterations: int):
        """
        Инициализация проверяющего. Задает коичество итераций проверки.
        """
        
        self._iterations = iterations
        
        
    def authenticate(self, client: FiatShamirClient, ca: FiatShamirCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        trace('[FiatShamirVerifier]', f'Attempt to authenticate client {client.identifier}')
        trace('[FiatShamirVerifier]', f'Number of iterations = {self._iterations}')
        
        is_successful = True
        
        for i in range(self._iterations):
            x = client.get_x(ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {x = }')
            
            e = randint(0, 1)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {e = }')
            
            y = client.get_y(e, ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {y = }')
            
            pk = ca.get_public_key(client.identifier)
            product = gmpy2.mul(x, gmpy2.powmod(pk, e, ca.modulus))
            
            successful_iteration = gmpy2.powmod(y, 2, ca.modulus) == gmpy2.t_mod(product, ca.modulus)
            is_successful &= successful_iteration
            
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] success: {successful_iteration}')
            
            if not successful_iteration:
                break
            
        print(f'Authentication successful: {is_successful}')
        return is_successful

----

In [8]:
# Создаю доверенный центр
ca = FiatShamirCA()

[TRACE] [FiatShamirCA] Modulus = 10565573616673509518022881196141051727740382579011175610416418713937825749489768983491071126331334365140842750085690869187901418205840495583420401508682036335174222847974808642104322523934461995917616091737059893768510538728373543350108200576493906376556489684564406444773641027267521642231042923903892720954380537591596399846985649664974985845237598490804967443413918161025158358457081550919908947361532900672048154579139647599401700901366565681253244187107060965788508770826812604570884548191788223529880045041582564585566113380675844768676219228363835190121275509850222334698154496598256095044325077226351650273251


In [9]:
# Создаю двух пользователей:
# - Алису - валидного пользователя
# - Еву - пользователя, представляющегося Алисой. но не знающего закрытого ключа Алисы
alice = FiatShamirClient(ca.modulus)
eve = FiatShamirClient(ca.modulus, 
                       fake_data=(alice.identifier, alice.public_key))

[TRACE] [FiatShamirClient] Client 5016f38c-b006-4d34-a66c-45d6a9008305
[TRACE] [FiatShamirClient] Public key = 8861658659025446714743172861290527737867853332919071962707980518999846139501483877813970280019114592078143412181632438742725650327230287463980218977489837242657474954872046121934094561107239570636045578215840139020359529794016942431230786713209433624284948384888433973672285640239016453077579056031424735934522667759481042108602650407457682031895038927214579820124312155919735804831258727032779156610786661579580361464866284665684475605884997779992177622203700013831737821970165924034021490889955568688151618452082078336860191952273071104629561482044912811704727263142547064376351094836224695917561490462337863675681
[TRACE] [FiatShamirClient] Client 5016f38c-b006-4d34-a66c-45d6a9008305
[TRACE] [FiatShamirClient] Public key = 88616586590254467147431728612905277378678533329190719627079805189998461395014838778139702800191145920781434121816324387427256503272302874639802189774898372426574

In [10]:
# Регистрирую валидного пользователя
ca.register_client(alice.identifier, alice.public_key)

[TRACE] [FiatShamirCA] Attempting to register client 5016f38c-b006-4d34-a66c-45d6a9008305 
        with public key 8861658659025446714743172861290527737867853332919071962707980518999846139501483877813970280019114592078143412181632438742725650327230287463980218977489837242657474954872046121934094561107239570636045578215840139020359529794016942431230786713209433624284948384888433973672285640239016453077579056031424735934522667759481042108602650407457682031895038927214579820124312155919735804831258727032779156610786661579580361464866284665684475605884997779992177622203700013831737821970165924034021490889955568688151618452082078336860191952273071104629561482044912811704727263142547064376351094836224695917561490462337863675681
[TRACE] [FiatShamirCA] Client registered successfully


In [11]:
# Для простоты будет 3 итерации
ITERATIONS = 3

# Создаю проверяющую сторону
verifier = FiatShamirVerifier(ITERATIONS)

In [12]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 5016f38c-b006-4d34-a66c-45d6a9008305
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(5175758888381996768294019539834348404099097115527707638467874302050400004547157770545694822125540434923497016053882767023821428293345205484422121024855754581343763660424268293432801817040597820462211779812535250394043406451623026808395864034870781873757461416332591406429045273317185889067768427860559140356497398476822030811298172039774065264125837696875102204606559427275944317718649683791541084218592815072270171344297599763589012127914496767429111671859836214888146651633694848910101730791967412994110011086224827638902468994138505009282362502663662297721704776411147400364312398866268909385678990920681698416680)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(240078972313950414209118375334206491704333643560546314792159989484194291513

True

In [13]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 5016f38c-b006-4d34-a66c-45d6a9008305
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(9291977411306403622653513028820187354422323590663040113558109697120674499676422162623003364844645294956027474440688719682171464258572059654146545430319032152442360936454140939921385020888773235168888399549627865215117435878872699643901649862773741235253282199544212108930738664210461541018300644137765255743430134294618873735360103284735679902593243757801331807828014549212183612449328232958610911898406200512973026665506296680860490299427256102846025153917534763065780392617998637872305128926093622040637727962915214175173455380900989925844974363585550626217914992981421515415602420585162478543764164689636274278746)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(673457468940660235362973281089816973977768456800154598888213571267121064976

False

---

## Протокол Шнорра
![Schnorr Protocol](./images/Schnorr_2.png)

In [14]:
class SchnorrCA(object):
    RSA_KEY_LENGTH = 4096
    
    def __init__(self):
        """
        Инициализация доверенного центра
        Генерация простых чисел p, q и элемента g, а также ключевой пары СА
        """
        
        self._db = {}
        
        #
        # Генерация параметров протокола
        #
        
        self._p, self._q, self._g = SchnorrCA._generate_parameters()
        
        #
        # Ключевая пара RSA для СА
        #
        
        self._private_key = RSA.generate(SchnorrCA.RSA_KEY_LENGTH)
        self._public_key = self._private_key.publickey()
        
        trace('[SchnorrCA]', f'RSA public key = {(self._public_key.e, self._public_key.n)}')
        
        
    def register_client(self, 
                        client_id: uuid.UUID, 
                        client_public_key: gmpy2.mpz) -> tuple[uuid.UUID, gmpy2.mpz, bytes]:
        """
        Регистрация клиента. Принимает на вход идентификатор клиента и его открытый ключ.
        Возвращает подписанный сервером сертификат.
        Выбрасывает исключение, если клиент уже зарегистрирован.
        """
        
        if client_id in self._db:
            trace('[SchnorrCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Создаю сертификат
        #
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        signature = pss.new(self._private_key).sign(h)
        
        certificate = (client_id, client_public_key, signature)
        trace('[SchnorrCA]', f'Certificate = {certificate}')
        
        #
        # Сохраняю его и возвращаю
        #
        
        self._db[client_id] = certificate
        return certificate
    
    
    def verify_certificate(self, certificate: tuple[uuid.UUID, gmpy2.mpz, bytes]) -> bool:
        """
        Проверка сертификата.
        Возвращает признак валидности сертификата.
        """
        
        client_id, client_public_key, signature = certificate
        
        if client_id not in self._db:
            trace('[SchnorrCA]', f"Client {client_id} doesn't exist")
            raise ValueError("Client doesn't exist")
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        verifier = pss.new(self.public_key)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа СА.
        """
        
        return self._public_key
    
    
    @property
    def parameters(self) -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Параметры протокола (p, q, g)
        """
        
        return self._p, self._q, self._g
        
        
    @staticmethod
    def _generate_parameters() -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Генерация параметров протокола (p, q, g).
        """
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = gmpy2.mpz(number.getPrime(Q_LENGTH))
        p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
        
        while not gmpy2.is_prime(p):
            p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
            
        #
        # Сгенерирую число, взаимно простое с p
        #
        
        tmp = getrandbits(P_LENGTH)
        while gmpy2.gcd(tmp, p) != 1:
            tmp = getrandbits(P_LENGTH)
        
        #
        # По малой теореме Ферма такой g будет иметь порядок q
        #
        
        g = gmpy2.powmod(tmp, (p - 1) // q, p)
        
        trace('[SchnorrCA]', f'{p = }, {q = }, {g = }')
        return p, q, g

In [15]:
class SchnorrClient(object):
    def __init__(self, parameters: tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz], *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz, tuple] = None):
        """
        Инициализация клиента. Генерирует идентификатор клиента,
        ключевую пару.
        Последний параметр необходим для демонстрации неуспешной аутентификации.
        """
        
        self._p, self._q, self._g = parameters
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        
        self._cert = None if fake_data is None else fake_data[2]
        self._r = None
        
        self._private_key = randint(1, self._q - 1)
        self._public_key  = gmpy2.powmod(
            self._g, -self._private_key, self._p) if fake_data is None else fake_data[1]
        
        trace('[SchnorrClient]', f'Client {self._id}')
        trace('[SchnorrClient]', f'Public key = {self._public_key}')
        
        
    def register(self, ca: SchnorrCA) -> None:
        """
        Регистрация клиента на СА. Принимает дескриптор СА.
        """
        
        self._cert = ca.register_client(self.identifier, self.public_key)
        
        
    def get_x(self) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = g ** r mod p
        где r - случайное целое число из отрезка [1, q - 1].
        Сохраняет значение r.
        """
        
        self._r = randint(1, self._q - 1)
        return gmpy2.powmod(self._g, self._r, self._p)
    
    
    def get_y(self, e, security_parameter) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = a * e + r mod q
        где r - сохраненное в get_x служебное значение, e - число,
        полученное от проверяющего, a - закрытый ключ клиента.
        """
        
        if e > 2 ** security_parameter or e < 1:
            trace('[SchnorrClient]', f'Invalid {e = }')
            raise ValueError('Invalid e')
            
        pre_result = gmpy2.mul(self._private_key, e) + self._r
        return gmpy2.t_mod(pre_result, self._q)
    
    
    @property
    def certificate(self):
        """
        Получение сертификата клиента.
        """
        
        return self._cert
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента.
        """
        
        return self._public_key
        
        
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id

In [16]:
class SchnorrVerifier(object):
    def __init__(self, parameters, security_parameter):
        """
        Инициализация проверяющего. Фактически только закрепляет параметры протокола.
        """
        
        self._p, self._q, self._g = parameters
        self._security_parameter = security_parameter
        
        assert security_parameter > 0 and 2 ** security_parameter < self._q
        
        self._e_values = set()
        
        
    def authenticate(self, client: SchnorrClient, ca: SchnorrCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        #
        # Получаю с клиента x и его сертификат
        #
        
        x, cert = client.get_x(), client.certificate
        trace('[SchnorrVerifier]', f'{x = }')
        
        #
        # Если сертификат битый или невалидный, то это плохо
        #
        
        if not ca.verify_certificate(cert):
            trace('[SchnorrVerifier]', f'Invalid certificate {cert}')
            return False
            
        #
        # Генерирую число е и отдаю его клиенту
        #
            
        e = randint(1, 2 ** self._security_parameter)
        while e in self._e_values:
            e = randint(1, 2 ** self._security_parameter)
            
        self._e_values.add(e)        
        trace('[SchnorrVerifier]', f'{e = }')
        
        y = client.get_y(e, self._security_parameter)
        trace('[SchnorrVerifier]', f'{y = }')
        
        #
        # Вычисляю значение, которое необходимо сравнить с полученным от клиента х
        # И сразу же сравниваю их
        # 
        
        product = gmpy2.mul(gmpy2.powmod(self._g, y, self._p), 
                            gmpy2.powmod(client.public_key, e, self._p))
        z = gmpy2.t_mod(product, self._p)
        trace('[SchnorrVerifier]', f'{z = }')
        
        is_successful = x == z
        print(f'Authentication successful: {is_successful}')
        
        return is_successful

----

In [17]:
# Создаю доверенный центр
ca = SchnorrCA()

[TRACE] [SchnorrCA] p = mpz(42492187716246303591873886351175132887413953214503963854531519429700190816582107989569043329482844927233580157457348315985559902261389793615227452066125336962917539408539288153169977724359243119816583843883813904978247032445130289743824391230066478441047274316416743672096667760880932921413121817786605084183), q = mpz(759667855392043910016840201278889082419369541309), g = mpz(38496797570154813156314799678668401887360033348834535930026231650880143771174023554915500051219122316406531684617681343057765441900121819664794692125046907190089803590692177428660158077393154395143341957965717146876611094921451520766361588663796434261503681376124746500822935346751804161790938133021977333977)
[TRACE] [SchnorrCA] RSA public key = (65537, 58602687443132883236807714956989548642399616096934404433023740678439887362550531315478029029607321854367900771644351423289585474980393212318430546960831471274566568799842014360580935575622993985738322547822266614649749255667092798911285131

In [18]:
# Создам Алису - валидного пользователя
alice = SchnorrClient(ca.parameters)

[TRACE] [SchnorrClient] Client bd7d8147-2e96-4cee-9a7d-bad133ae7635
[TRACE] [SchnorrClient] Public key = 11889122677074424677506345760468329142477273695440611736637641479923364805195270147088967274953342324554656171690519084934564536401546679955110274898699910445779949497303078831260010159175717604551028692486548640156489108747400405131799197572006430565008527250716999004710098983153811877207652306045753743591


In [19]:
# Регистрирую валидного пользователя
alice.register(ca)

[TRACE] [SchnorrCA] Certificate = (UUID('bd7d8147-2e96-4cee-9a7d-bad133ae7635'), mpz(11889122677074424677506345760468329142477273695440611736637641479923364805195270147088967274953342324554656171690519084934564536401546679955110274898699910445779949497303078831260010159175717604551028692486548640156489108747400405131799197572006430565008527250716999004710098983153811877207652306045753743591), b'\x06\xd3\x19eI\x1a.8\xcb\xed*\xe9&\xca\xc7nG9^^\xe4:$t`\xfb>\x11\xd1\xe1O\xf4\xc8zvS\xbb E8\xa4V\xd4\xb777\xcf!\x02\xa2Wqw\xce\xbf\xdc\x0es\xd0\xef\xbc\x17\xc4hh\xb0\xb8\'\xb1\xf8\xf2\xff51\xa8\x7f\x97T\x0f\x06"\xfeN\x18^%+\xf5B\xe2L\xb1\xf6X\xcdF\xe2\xc6\xbcHfE0G\xd6<\x15\xb8\xb6\x94\x1d\xb3\xbc\xac\xa0\xd1%\x17.\xd9\xf7\x8a\xca\x05\xfd\xec\xdc\xca\xa5o\xe7\xc1\xad\xcf\xef\xdc\n\x11\xa9u\x1f\x14\xef\x8cKi\x0b*\x93-\x0cQ\xf4\x94b\x08\xec\x83\x86\xd0I\x87\x97\xfa@\xbea\x18\xbe)W\xf54\xff@H\xa66t\xc9\xba\xcc\x1f\x16\x0eR\xb5#V\xe8z%\x8dVc\x7f\xf0X\xa5\xeb\xae\xa8\x86j\x07\xcd\\\x86\x9a\x93\x917\xf

In [20]:
# Параметр безопасности положим равным 10 для демонстрации
security_parameter = 10

# Создаю проверяющую сторону
verifier = SchnorrVerifier(ca.parameters, security_parameter)

In [21]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [SchnorrVerifier] x = mpz(33209829322026888714222930572616777954978472216202647915615056224752511348556424047152003874179274952510359062951926848846530536598662269611430864159226128367083810297903322607377345769920939338554368411658085900826802311555869507983725791375269142028550731241462711302470425844203864445682303930118045445730)
[TRACE] [SchnorrVerifier] e = mpz(933)
[TRACE] [SchnorrVerifier] y = mpz(173991743477541425087934729160906597071791526788)
[TRACE] [SchnorrVerifier] z = mpz(33209829322026888714222930572616777954978472216202647915615056224752511348556424047152003874179274952510359062951926848846530536598662269611430864159226128367083810297903322607377345769920939338554368411658085900826802311555869507983725791375269142028550731241462711302470425844203864445682303930118045445730)
Authentication successful: True


True

In [22]:
# Создаю Еву - клиента, который пытается представиться Алисой
eve = SchnorrClient(ca.parameters, 
                    fake_data=(alice.identifier, alice.public_key, alice.certificate))

[TRACE] [SchnorrClient] Client bd7d8147-2e96-4cee-9a7d-bad133ae7635
[TRACE] [SchnorrClient] Public key = 11889122677074424677506345760468329142477273695440611736637641479923364805195270147088967274953342324554656171690519084934564536401546679955110274898699910445779949497303078831260010159175717604551028692486548640156489108747400405131799197572006430565008527250716999004710098983153811877207652306045753743591


In [23]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [SchnorrVerifier] x = mpz(29774271887083324961343510679709702844439407169277279947600561198662953394398868439060349650416497706416649837360456505693545728413062035109485970737996020045791790433976796257183015270428061242407299712453448966056988775507671615926059072518358448725356420959218479113017811043356859793298711562914321143312)
[TRACE] [SchnorrVerifier] e = mpz(520)
[TRACE] [SchnorrVerifier] y = mpz(21365846562367195232212684665749383175012289796)
[TRACE] [SchnorrVerifier] z = mpz(18488921263156169410758695705536906893178310501674731177757481137657296020196094417490495876609678335570938259513726870924293430763370131888985244711528646792251042587747048034771591064908047926994804109271547270103243626584769400774690722812978100066600275551315081683134499685073318082155700458187119941683)
Authentication successful: False


False